# Scrape HN

This file scrapes this month's HN Jobs post using Selenium

In [56]:
#| default_exp scraper

#| export
who_is_hiring = "https://news.ycombinator.com/item?id=40846428"


In [57]:
%pip install -q selenium
%pip install -q nbdev
%pip install -q webdriver-manager
%pip install -q pillow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [58]:
#| export
from nbdev.showdoc import *

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [59]:
#| export
import io
from PIL import Image

dataLocation = "previousData/"

def getPosts(postsHtml: list):
    jobPosts = []

    for post in postsHtml:
        indent = post.find_element(By.CLASS_NAME, "ind").get_attribute("indent")

        # ignore reply comments: comments indented by 1+
        # later on it might be useful to attach these to the parent comment
        if (indent == "0"):

            try:
                hnuser = post.find_element(By.CLASS_NAME, "hnuser").text
            except NoSuchElementException:
                hnuser = ''  # or a default value

            try:
                date = post.find_element(By.CLASS_NAME, "age").get_attribute("title")
            except NoSuchElementException:
                date = 0  # or a default value

            try:
                comment = post.find_element(By.CLASS_NAME, "comment").text
            except NoSuchElementException:
                comment = ''  # or a default value

            name = date + "_" + hnuser
            fileName = dataLocation + 'images/'+name+".png"
            # image_binary = post.screenshot_as_png 
            # img = Image.open(io.BytesIO(image_binary))
            # img.save(fileName)

            jobPost = {
                "hnuser": hnuser,
                "date": date,
                "comment": comment,  
                "fileName": fileName,
                "html": post.get_attribute("outerHTML")
            }

            jobPosts.append(jobPost)

    return jobPosts

In [3]:
#| export

from os import wait


def hnJobsPaginated(url, page: int):
    return f'{url}&p={page}'

def scrape(url: str):
    url = url or who_is_hiring

    #create an instance of the Service object
    service = Service(executable_path=ChromeDriverManager().install())
    #start Chrome using the service keyword
    driver = webdriver.Chrome(service=service)

    def scrapePage(url: str, page: int = 1):
        print(f"Scraping page {page}")

        paginatedUrl = hnJobsPaginated(url, page)
        driver.get(paginatedUrl)

        tbody = driver.find_element(By.CLASS_NAME, "comment-tree")
        postsHtml = tbody.find_elements(By.CLASS_NAME, "athing")
        posts = getPosts(postsHtml)

        try:
            driver.find_element(By.CLASS_NAME, "morelink")
            page += 1
            posts.append(scrapePage(url, page))
        except NoSuchElementException:
            print('all pages scraped')

        return posts    

    posts = scrapePage(url)

    print(f"Scraped {len(posts)} posts")

    driver.quit()

    return posts

In [61]:
#| export
import json

def savePosts(filename, posts):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(posts, f, ensure_ascii=False, indent=4)

In [62]:
#| export
import json

def loadJSON(filename):
    posts = False
    try: 
        with open(filename, 'r', encoding='utf-8') as f:
            posts = json.load(f)
    finally:
        return posts

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()
